## Install dependencies

In [ ]:
!git clone https://github.com/eyldlv/DERMIT-Corpus.git

Cloning into 'DERMIT-Corpus'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 117 (delta 5), reused 23 (delta 5), pack-reused 92
Receiving objects: 100% (117/117), 87.56 MiB | 16.47 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (85/85), done.


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.1 MB/s eta 0:00:00
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-py3-none-any.whl size=16863 sha256=a0fc461d08de59ad8f1dcda99530d7467b7518369981f2a28c92e4756bf5b656
  Stored in directory: /root/.cache/pip/wheels/f6/48/c4/942f7a1d556fddd2348cb9ac262f251873dfd8a39afec5678e
Successfully built sqlitedict
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's de

In [ ]:
from pathlib import Path

import transformers
from transformers import AutoTokenizer, AutoModel

from evaluation.romansh_alignment.encoders.hf import HuggingfaceEncoder
from evaluation.romansh_alignment.word_aligners.simalign_aligner import SimalignAligner
from evaluation.romansh_alignment.word_alignment import WordAlignmentBenchmark
from evaluation.romansh_alignment.sentence_alignment import SentenceAlignmentBenchmark

## Load model

In [ ]:
model_name = "ZurichNLP/swissbert"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at ZurichNLP/swissbert were not used when initializing XmodModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XmodModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XmodModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XmodModel were not initialized from the model checkpoint at ZurichNLP/swissbert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = model.to("cuda:0")

In [ ]:
de_lang_id = 0  # de_CH
rm_lang_id = 3  # rm_CH

## Evaluate sentence alignment

In [ ]:
sentence_benchmark = SentenceAlignmentBenchmark()

Removing 3 duplicate sentences:
  Die Wanderausstellung " Rumantsch è… " ist ein Projekt des Kantons Graubünden .
  L' exposiziun ambulanta " Rumantsch è… " è in project dal chantun Grischun .

  Grillfeuer sind mit grosser Vorsicht und vorzugsweise in festeingerichteten Feuerstellen zu entfachen .
  Fieus per grillar duain vegnir envidads cun gronda precauziun e da preferenza en fuclars fixs .

  Seit mehreren Wochen hat es auf der Alpensüdseite keine nennenswerten Niederschläge gegeben .
  Dapi pliras emnas n' hai dà naginas precipitaziuns pli grondas pli al sid da las Alps .



In [ ]:
encoder = HuggingfaceEncoder(
    model=model,
    tokenizer=tokenizer,
    aggregation="average",
)

In [ ]:
sentence_benchmark.evaluate_encoder_bert_score(encoder, batch_size=600, src_lang_id=de_lang_id, tgt_lang_id=rm_lang_id)

Encoding tokens with HuggingfaceEncoder(ZurichNLP_swissbert, aggregation=average)...


100%|██████████| 597/597 [00:36<00:00, 16.17it/s]


Calculating accuracy...


100%|██████████| 597/597 [47:20<00:00,  4.76s/it]


Accuracy DE -> RM: 0.9564489112227805


SentenceAlignmentResult(num_sentences=597, accuracy=0.9564489112227805)

## Evaluate word alignment

In [ ]:
word_benchmark = WordAlignmentBenchmark()

Removing 3 duplicate sentences:
  Die Wanderausstellung " Rumantsch è… " ist ein Projekt des Kantons Graubünden .
  L' exposiziun ambulanta " Rumantsch è… " è in project dal chantun Grischun .

  Grillfeuer sind mit grosser Vorsicht und vorzugsweise in festeingerichteten Feuerstellen zu entfachen .
  Fieus per grillar duain vegnir envidads cun gronda precauziun e da preferenza en fuclars fixs .

  Seit mehreren Wochen hat es auf der Alpensüdseite keine nennenswerten Niederschläge gegeben .
  Dapi pliras emnas n' hai dà naginas precipitaziuns pli grondas pli al sid da las Alps .



In [ ]:
aligner = SimalignAligner(
    tokenizer=tokenizer,
    model=model,
    method="match",
    aggregation="average",
)

In [ ]:
result = word_benchmark.evaluate(aligner, src_lang_id=de_lang_id, tgt_lang_id=rm_lang_id)
result

100%|██████████| 597/597 [00:45<00:00, 13.06it/s]


WordAlignmentResult(aer=0.13571533598000296, precision=0.8803354800059907, recall=0.8488086642599278, f1=0.8642846640199972)